In [ ]:
### REFERENCES : 
# http://initd.org/psycopg/docs/usage.html#passing-parameters-to-sql-queries
# https://trac.edgewall.org/wiki/TracDev/DatabaseApi#RulesforDBAPIUsage
# http://aklaver.org/wordpress/2018/04/21/building-dynamic-sql-using-psycopg2/
# https://stackoverflow.com/questions/13793399/passing-table-name-as-a-parameter-in-psycopg2

In [25]:
import psycopg2
from psycopg2 import Error, sql
import pandas as pd

In [33]:
### Parametes:
# Database : 'db1'
# User : 'postgres'
# password : 'post'

# Note : Default port is usually 5432

def connect():
    
    '''
    Returns a connection to the database.
    '''
    
    try:
        
        # Connects to database 'db1'
        connection = psycopg2.connect(
            dbname="db1", 
            user="postgres", 
            host="localhost", 
            password="post", 
            port="5433"
        )
        return connection
    
    except:
        print("Unable to connect to database")

In [34]:
def create_table():
    
    '''
    Creates a table - 'databases' with columns:
        - event : char(100)
        - path : char(100)
        - value : decimal
    taking (event,path) as primary key.
    '''

    try:
        connection = connect()
        cursor = connection.cursor()

        cursor.execute( 
                """
                CREATE TABLE databases(
                    event CHAR(100) NOT NULL,
                    path CHAR(100) NOT NULL,
                    value DECIMAL,
                    PRIMARY KEY( event, path)
                ); 
                """
        )

        connection.commit()
        print("Table created")

    except (Exception, psycopg2.DatabaseError) as error :
        print ("Error : ", error)

    finally:
        if(connection):
            cursor.close()
            connection.close()

In [35]:
def insert_into_table(event,path,value):
    
    '''
    Inserts a row into the 'databases' table
    with the provided values
    '''
    
    try:
        connection = connect()
        cursor = connection.cursor()
        cursor.execute("INSERT INTO databases (event,path,value) VALUES (%s, %s, %s);",(event, path, value))        
        connection.commit()

    except (Exception, psycopg2.DatabaseError) as error :
        print ("Error : ", error)

    finally:
        if(connection):
            cursor.close()
            connection.close()

In [36]:
def query1(term):
    
    '''
    Queries the 'databases' relation and return rows 
    that (partially) contain the provided 'term'
    
    Returns a Pandas Dataframe containing the rows
    '''
    
    record = None    
    term = '%' + term + '%'
    
    try:
        connection = connect()
        cursor = connection.cursor()

        cursor.execute("SELECT * FROM databases WHERE path LIKE %s ESCAPE ''", (term,))        
        record = cursor.fetchall()
        record = pd.DataFrame(record, columns=['event','path','value'])
        
    except (Exception, psycopg2.DatabaseError) as error :
        print ("Error : ", error)

    finally:
        if(connection):
            cursor.close()
            connection.close()
    
    return record

In [37]:
def query2(path=None, comparator='<', value='1'):
    
    '''
    Queries the 'databases' relation and return rows wherein 
        - the path matches with the 'path' provided
        - values satisfies the condition specified by 'comparator'/'value' pair. 
        
    Path must be string
    Allowed Comparators : {'=', '<', '>', '<=', '>='}
    Value must be Int/Float
    
    Returns a Pandas Dataframe containing the rows
    '''
    
    record = None 
    
    ### Validate passed parameters to prevent injection attacks ###
    
    # Check for proper comparator
    allowed_comparators = {'=', '<', '>', '<=', '>='}
    
    if(comparator not in allowed_comparators) : 
        print("Error : Invalid comparator provided, choose from ", allowed_comparators)
        return record
    
    # Check for proper value
    try:
        test_value = float(value)
    except:
        print("Error : Invalid int/float value provided")
        return record

    # Check for proper path
    if(path != None):
        try:
        
            connection = connect()
            cursor = connection.cursor()
            cursor.execute("SELECT * from databases WHERE path = %s", (path,))
            record = cursor.fetchall()
            if(len(record) == 0):
                print("Error : Path provided does not corrospond to valid entry")
                return record
         
        except (Exception, psycopg2.DatabaseError) as error :
            print("Error : ", error)

        finally:
            if(connection):
                cursor.close()
                connection.close()

    ### Execute query after validation ###
    
    try:
        connection = connect()
        cursor = connection.cursor()
        
        sql = "SELECT * FROM databases WHERE value " + comparator + " " + str(value)
        if(path != None): 
            path = "'" + path + "'"
            sql = sql + " AND path = " + path
        
        sql = sql + ';'
        cursor.execute(sql)        
        
        record = cursor.fetchall()
        record = pd.DataFrame(record, columns=['event','path','value'])
        
    except (Exception, psycopg2.DatabaseError) as error :
        print ("Error : ", error)

    finally:
        if(connection):
            cursor.close()
            connection.close()
    
    return record

In [50]:
# Create the table
create_table()

Table created


In [51]:
# Insert values into the table

insert_into_table("event_1", "path_1", 0.15)
insert_into_table("event_1", "path_2", 1.01)
insert_into_table("event_1", "path_3", 1.52)

insert_into_table("event_2", "path_1", 1.12)
insert_into_table("event_2", "path_3", 0.55)
insert_into_table("event_2", "path_4", 9.51)

insert_into_table("event_3", "path_1", 1.11)
insert_into_table("event_3", "path_5", 6.55)
insert_into_table("event_3", "path_8", 4.31)

insert_into_table("event_4", "path_2", 6.12)
insert_into_table("event_4", "path_5", 0.01)

insert_into_table("event_5", "path_7", 8.21)

In [59]:
# query the table for paths containing the string '_1'

out1 = query1(term="_1")
out1

,event,path,value
0,event_1 ...,path_1 ...,0.15
1,event_2 ...,path_1 ...,1.12
2,event_3 ...,path_1 ...,1.11


In [60]:
# Query the table for events that match the specified conditions.

out2 = query2(path='path_1', comparator='>=', value='1')
out2

,event,path,value
0,event_2 ...,path_1 ...,1.12
1,event_3 ...,path_1 ...,1.11
